In [1]:
# Imports
import cv2
import numpy as np
import math
import pandas as pd


In [ ]:
def distance(p1: tuple[int, int], p2: tuple[int, int]) -> float:
    dist = math.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2) # distance formula
    return(dist)

In [ ]:
def editors(sample):
    # Makes it one channel
    gray = cv2.cvtColor(sample, cv2.COLOR_BGR2GRAY)

    # Closing operation 
    kernel = np.ones((5, 5), np.uint8)
    img = cv2.morphologyEx(gray, cv2.MORPH_CLOSE, kernel, iterations= 25)

    # Filtering and fixing
    img = cv2.GaussianBlur(img, (11, 11), 0)
    ret, thresh = cv2.threshold(img, 140, 225, cv2.THRESH_BINARY) 
    
    #Finding Canny Lines
    canny = cv2.Canny(thresh, 0, 200)
    canny = cv2.dilate(canny, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)))
    canny = np.float32(canny) 
    return(canny)

In [ ]:
def cornerLexy(canny):
    dst = cv2.cornerHarris(canny , 5, 3, 0.04)
    ret, dst = cv2.threshold(dst, 0.1*dst.max(), 255, 0)
    dst = np.uint8(dst)
    ret, labels, stats, centroids = cv2.connectedComponentsWithStats(dst)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
    corners = cv2.cornerSubPix(canny, np.float32(centroids),(5, 5), (-1, -1), criteria)
    return(corners)

In [ ]:
def corner_sort(sample,corners):
    w = sample.shape[1]
    h = sample.shape[0]

    df = pd.DataFrame(data = corners)
    halfwayx = min(df[0]) + ((max(df[0]) - min(df[0])) / 2)
    halfwayy = min(df[1]) + ((max(df[1]) - min(df[1])) / 2)

    botl = [item for item in corners if item[0] <= halfwayx and item[1] <= halfwayy]
    topl = [item for item in corners if item[0] <= halfwayx and item[1] >= halfwayy]
    botr = [item for item in corners if item[0] >= halfwayx and item[1] <= halfwayy]
    topr = [item for item in corners if item[0] >= halfwayx and item[1] >= halfwayy]

    dist1 = [distance(x, (0, 0)) for x in botl]
    dist2 = [distance(x, (0, h)) for x in topl]
    dist3 = [distance(x, (w, 0)) for x in botr]
    dist4 = [distance(x, (w, h)) for x in topr]

    bl = botl[dist1.index(min(dist1))]
    tl = topl[dist2.index(min(dist2))]
    br = botr[dist3.index(min(dist3))]
    tr = topr[dist4.index(min(dist4))]

    srccorners = np.array([bl, br, tr, tl], np.float32)
    return(bl,br,tl,tr,srccorners)

In [ ]:
def gather(bl, br, tl, tr):
    widtha = distance(bl, br)
    widthb = distance(tl, tr)
    mW = int(max(widtha, widthb))

    heighta = distance(br, tr)
    heightb = distance(bl, tl)
    mH = int(max(heighta, heightb))

    destcorners = np.array([(0, 0), (mW, 0), (mW, mH), (0, mH)], np.float32)
    return(mW,mH,destcorners)

In [ ]:
def persp_match(sample, srccorners, destcorners, mW, mH):
    matrix = cv2.getPerspectiveTransform(srccorners, destcorners)
    result = cv2.warpPerspective(sample, matrix, (mW, mH))
    return(result)

In [ ]:
def cleaning(result):
    #Thresholding 
    gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)
    ret, thresh3 = cv2.threshold(gray, 127, 225, cv2.THRESH_BINARY)

    #Removing Noise
    img2 = cv2.GaussianBlur(thresh3, (11, 11), 0)
    canny2 = cv2.Canny(img2, 0, 200)
    return(canny2)

In [ ]:
def boxer(result,canny2,mW,mH):
    center = []
    rectangles = []
    contours = cv2.findContours(canny2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[0 if len(contours) == 2 else 1]
    for cntr in contours:
        x, y, w, h = cv2.boundingRect(cntr)
        if (w*h >= mW*mH*0.00028) and (w + h <= (mW+mH)*0.1):
            cv2.rectangle(result, (x, y), (x + w, y + h), (0, 0, 255), 2)
            rectangles.append([x, y, w, h])
            center.append((x + (0.5*w), y + (0.5*h)))
    return(center)

In [ ]:
def cartographer(result,mW,mH):
    #Put plotting lines into comments so it could be faster, turn back on to see grid lines
    #Finding the grid lines
    #dx = int((1 / 12) * mW)
    #dy = int((1 / 8.4) * mH)
    #grid_color = [255, 0, 0]
    #result[::dy,:,:] = grid_color
    #result[:,::dx,:] = grid_color

    #Plot
    #plt.imshow(result)
    #plt.show()

    #Value of those gridlines
    x_bounds = np.arange(0, mW, (1 / 12)*mW).tolist()
    y_bounds = np.arange(0, mH, (1 / 8.4)*mH).tolist()
    return(x_bounds,y_bounds)

In [ ]:
def sorter(center,x_bounds,y_bounds):
    #Separating the coordinates into x and y
    cx = [item[0] for item in center]
    cy = [item[1] for item in center]

    #Grid numbering
    gridx = [sum(1 for b in x_bounds if b < x) for x in cx]
    gridy = [sum(1 for b in y_bounds if b < y) for y in cy]
    return(gridx,gridy)

In [ ]:
def scanner(imagename):
    #Adding the pictures
    sample = cv2.imread(imagename, cv2.IMREAD_UNCHANGED)
    #All of the things
    canny = editors(sample)
    corners = cornerLexy(canny)
    bl,br,tl,tr,srccorners = corner_sort(sample,corners)
    mW,mH,destcorners = gather(bl,br,tl,tr)
    result = persp_match(sample, srccorners, destcorners, mW, mH)
    canny2 = cleaning(result)
    center = boxer(result,canny2,mW,mH)
    x_bounds, y_bounds = cartographer(result,mW,mH)
    gridx,gridy = sorter(center,x_bounds,y_bounds)
    return(gridx,gridy)

All the functions and what they do: <br>

editors - cleans raw file, returns a canny image of the screen without other background elements <br>
cornerLexy - determines the corners from the canny image, returns a list of (x,y) corners <br>
corner_sort - determines the corners closest to the edges of the image, returns the coordinates of those real corners and srccorners <br>
gather - uses the real corner coordinates to find how wide and tall the screen is in the image, returns max width/height and destcorners <br>
persp_match - uses srccorners and destcorners, makes the image flat and forward facing to fix any camera tilt/turn, returns an adjusted image <br>
cleaning - cleans the content on screen, removes pixelated lines that come with taking a pic of a screen, thresholds so the creatures are black <br>
boxer - puts boxes around the black images, calcluates the center of those boxes and returns that as a list of center coordinates (x,y) <br>
cartographer - takes the cleaned picture, grids it proportionate to the size, returns the grid line locations for x and y <br>
sorter - cycles each center point through the list of grid lines to find which sector it lies in, returns the x sector and y sector of each <br>
scanner - runs the sample through all of these

Need to submit 5 images, one for each of the frames. Picture should not have glare, not on a white background if possible, and without joycons in the frame. 

In [ ]:


#sample = cv2.imread('IMG_2233.jpg', cv2.IMREAD_UNCHANGED)

In [ ]:
# something that allows users to upload 5 pictures
# maybe it requests user to do it in a specific order

images = ['IMG_2231.jpg', 'IMG_2232.jpg', 'IMG_2207.jpg', 'fish2test.jpg', 'IMG_2233.jpg']
collection_df = pd.DataFrame()
frames = ('bug1', 'bug2', 'fish1', 'fish2', 'sea')

for sample in images:
    add_df = pd.DataFrame()
    gridx, gridy = scanner(sample)
    add_df['X'] = gridx
    add_df['Y'] = gridy
    add_df['Frame'] = frames[images.index(sample)]
    collection_df = pd.concat([collection_df, add_df], ignore_index= True, axis=0)



In [ ]:
#Comparing what we have vs the pedia 
pediadf = pd.read_csv('Pedia.csv')

#List with indicator, left-only means missing
user_collection = pediadf.merge(collection_df.drop_duplicates(), on=['X','Y','Frame'], how='left', indicator=True)

In [ ]:
# Users should be able to select their hemisphere, view/add/remove critters from their collection, see what they haven't caught
# that they can catch at that date/time, see what date/time they'd be able to catch the most new critters, change the date/time
# filter their collections to see caught/uncaught, maybe see percent done

In [2]:
cap = cv2.VideoCapture(0)

while True:
    capture = False
    success, frame = cap.read()
    cv2.imshow('live', frame)
    if cv2.waitKey(1) & 0xFF ==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
